### self attention

### 3.1 simple self-attention 

In [1]:
import torch

inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your     (x^1)
    [0.55, 0.87, 0.66], # journey  (x^2)
    [0.57, 0.85, 0.64], # starts   (x^3)
    [0.22, 0.58, 0.33], # with     (x^4)
    [0.77, 0.25, 0.10], # one      (x^5)
    [0.05, 0.80, 0.55]] # step     (x^6)
)

In [2]:
input_query = inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [3]:
input_1 = inputs[0]
input_1

tensor([0.4300, 0.1500, 0.8900])

In [4]:
torch.dot(input_query, input_1)

tensor(0.9544)

In [5]:
# dot product from scratch

res = 0.
i =  1
for idx, element in enumerate(inputs[i]):
    res += inputs[i][idx] * input_query[idx]
    print(res)

res

tensor(0.3025)
tensor(1.0594)
tensor(1.4950)


tensor(1.4950)

In [6]:
i = 1
res = torch.dot(input_query, inputs[i])
res

tensor(1.4950)

In [7]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [8]:
att_weights_2_tmp = attn_scores_2/attn_scores_2.sum()

att_weights_2_tmp

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])

In [9]:
def softmax_naive(x):
    return torch.exp(x)/torch.exp(x).sum(dim = 0)


softmax_naive(attn_scores_2)


tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [10]:
attn_weights_2 = torch.softmax(attn_scores_2, dim = 0)
attn_weights_2


tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [11]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 把简单注意力机制应用到所有的context上

In [12]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j) 

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [13]:
attn_scores = inputs @ inputs.T
attn_scores

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])

In [14]:
attn_weights = torch.softmax(attn_scores, dim = 1)
attn_weights

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])

In [15]:
attn_weights.sum(dim = 1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])

In [16]:
all_contexts_vecs = attn_weights @ inputs
all_contexts_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

In [17]:
# 使用矩阵运算计算所有context
attn_scores = inputs @ inputs.T
attn_weights = torch.softmax(attn_scores, dim = 1)

all_contexts_vecs = attn_weights @ inputs
all_contexts_vecs

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])

### 建立真正可训练注意力权重

In [18]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [19]:
torch.manual_seed(123)

W_q = torch.nn.Parameter(torch.randn(d_in, d_out))
print(W_q)
W_k = torch.nn.Parameter(torch.randn(d_in, d_out))
print(W_k)
W_v = torch.nn.Parameter(torch.randn(d_in, d_out))
print(W_v)
W_q


Parameter containing:
tensor([[-0.1115,  0.1204],
        [-0.3696, -0.2404],
        [-1.1969,  0.2093]], requires_grad=True)
Parameter containing:
tensor([[-0.9724, -0.7550],
        [ 0.3239, -0.1085],
        [ 0.2103, -0.3908]], requires_grad=True)
Parameter containing:
tensor([[ 0.2350,  0.6653],
        [ 0.3528,  0.9728],
        [-0.0386, -0.8861]], requires_grad=True)


Parameter containing:
tensor([[-0.1115,  0.1204],
        [-0.3696, -0.2404],
        [-1.1969,  0.2093]], requires_grad=True)

In [20]:
query_2 = x_2 @ W_q
query_2


tensor([-1.1729, -0.0048], grad_fn=<SqueezeBackward4>)

In [21]:
keys = inputs @ W_k
values = inputs @ W_v

keys

tensor([[-0.1823, -0.6888],
        [-0.1142, -0.7676],
        [-0.1443, -0.7728],
        [ 0.0434, -0.3580],
        [-0.6467, -0.6476],
        [ 0.3262, -0.3395]], grad_fn=<MmBackward0>)

In [22]:
key_2 = keys[1]
print(key_2)

attn_scores_22 = torch.dot(query_2, key_2)
attn_scores_22

tensor([-0.1142, -0.7676], grad_fn=<SelectBackward0>)


tensor(0.1376, grad_fn=<DotBackward0>)

In [23]:
attn_scores_2 = query_2 @ keys.T
attn_scores_2

tensor([ 0.2172,  0.1376,  0.1730, -0.0491,  0.7616, -0.3809],
       grad_fn=<SqueezeBackward4>)

In [24]:
d_k = keys.shape[1]

atten_weights_2 =  torch.softmax(attn_scores_2/d_k**0.5, dim = 0)
atten_weights_2


tensor([0.1704, 0.1611, 0.1652, 0.1412, 0.2505, 0.1117],
       grad_fn=<SoftmaxBackward0>)

In [25]:
atten_weights_2.sum(dim = 0)

tensor(1.0000, grad_fn=<SumBackward1>)

In [26]:
context_vec_2 = atten_weights_2 @ values

context_vec_2

tensor([0.2854, 0.4081], grad_fn=<SqueezeBackward4>)

### implementing a compact self attention class

In [42]:
import torch.nn as nn

class SelfAttention_V1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_q = nn.Parameter(torch.randn(d_in, d_out))
        self.W_k = nn.Parameter(torch.randn(d_in, d_out))
        self.W_v = nn.Parameter(torch.randn(d_in, d_out))

    def forward(self, x):
        queries = x @ self.W_q
        keys = x @ self.W_k
        values = x @ self.W_v

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores/d_k**0.5, dim = 1)
        # print(attn_weights)
        # print(values)
        context_vec = attn_weights @ values

        return context_vec
        

In [43]:
torch.manual_seed(123)
sa_v1 = SelfAttention_V1(d_in = 3, d_out = 2)

sa_v1(inputs)

tensor([[0.2845, 0.4071],
        [0.2854, 0.4081],
        [0.2854, 0.4075],
        [0.2864, 0.3974],
        [0.2863, 0.3910],
        [0.2860, 0.4039]], grad_fn=<MmBackward0>)

In [48]:
class SelfAttention_V2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.W_q = torch.nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_k = torch.nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_v = torch.nn.Linear(d_in, d_out, bias = qkv_bias)

    def forward(self, x):
        queries = self.W_q(x)
        keys = self.W_k(x)
        values = self.W_v(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores/d_k**0.5, dim = 1)
        # print(attn_weights)
        # print(values)
        context_vec = attn_weights @ values

        return context_vec

In [49]:
torch.manual_seed(123)
sa_v1 = SelfAttention_V2(d_in = 3, d_out = 2)

sa_v1(inputs)

tensor([[-0.5337, -0.1051],
        [-0.5323, -0.1080],
        [-0.5323, -0.1079],
        [-0.5297, -0.1076],
        [-0.5311, -0.1066],
        [-0.5299, -0.1081]], grad_fn=<MmBackward0>)

### 使用casual attention mask